<a href="https://colab.research.google.com/github/Penetrati0n/fuzzy-sets-calculator/blob/main/fuzzy_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

In [1]:
import numpy as np
import pandas as pd

import IPython
from IPython.core import display as ICD

def parse_sets(data: str):
    global SETS
    lines = data.strip().split('\n')
    for line in lines:
        SETS[line[0]] = np.array([float(e) for e in line.replace(',', '.').split('\t')[1:]])

def get_set(set_name: str):
    global SETS
    return SETS[set_name.upper()]

# Абсолютное линейное расстояние.
def ALR(set1, set2):
    return np.sum(np.abs(set1 - set2))

# Абсолютное квадратичное расстояние.
def AKR(set1, set2):
    return np.sqrt(np.sum(np.power(set1 - set2, 2)))

# Относительное линейное расстояние.
def OLR(set1, set2):
    return 1 / set1.shape[0] * ALR(set1, set2)

# Относительное квадратичное расстояние.
def OKR(set1, set2):
    return 1 / np.sqrt(set1.shape[0]) * AKR(set1, set2)

# Абсолютная евклидова норма.
def AEN(set1, set2):
    return np.power(AKR(set1, set2), 2)

# Относительная евклидова норма.
def OEN(set1, set2):
    return np.power(OKR(set1, set2), 2)

#--------------------------------------------------------------------------------

def parse_sets_pairs(data: str) -> list:
    return [e.strip()[0] + e.strip()[-1] for e in data.strip().split(',')]

# Задание 1.
def task1(set_pairs: list):
    df = pd.DataFrame(columns=['НМ', 'АЛР', 'ОЛР', 'АКР', 'ОКР', 'АЕН', 'ОЕН'])
    sets_keys = list(SETS.keys())
    for i in range(len(SETS) - 1):
        set1 = SETS[sets_keys[i]]
        for j in range(i + 1, len(SETS)):
            set2 = SETS[sets_keys[j]]
            if sets_keys[i] + sets_keys[j] in set_pairs:
                df.loc[len(df.index)] = [f'{sets_keys[i]} и {sets_keys[j]}', ALR(set1, set2), OLR(set1, set2), 
                                        AKR(set1, set2), OKR(set1, set2), AEN(set1, set2), OEN(set1, set2)]
            else:
                df.loc[len(df.index)] = [f'{sets_keys[i]} и {sets_keys[j]}', -1, -1, -1, -1, -1, -1]
    df = df.set_index('НМ')
    questions = df[['АЛР', 'АКР']].copy()
    for col in questions.columns:
        questions[col] = np.where(questions[col] == questions[col].max(), 1, 0)
    return df, questions
    # return df, df[df['АЛР'] == df['АЛР'].max()].index[0], df[df['АКР'] == df['АКР'].max()].index[0]

#--------------------------------------------------------------------------------

def common_set(set1):
    return np.where(set1 <= 0.5, 0, 1)

def liner_index(set1):
    return 2 / set1.shape[0] * ALR(set1, common_set(set1))

def liner_index2(set1):
    return 2 / set1.shape[0] * np.sum(np.minimum(set1, 1 - set1))

def rect_index(set1):
    return 2 / np.sqrt(set1.shape[0]) * AKR(set1, common_set(set1))

def entropy(set1):
    tmp = set1 / np.sum(set1)
    return -1 / np.log(set1.shape[0]) * np.sum(tmp * np.log(tmp))

# Мера четкость по Ягеру.
def clarity_yager(set1, set2, p: int):
    return 1 / set1.shape[0] * np.power(np.sum(np.power(np.abs(set1 - set2), p)), 1 / p)

# Мера нечеткость по Ягеру.
def fuzzy_yager(set1, set2, p: int):
    return 1 - clarity_yager(set1, set2, p) / np.power(set1.shape[0], 1 / p) 

#--------------------------------------------------------------------------------

def parse_sets_names(data) -> list:
    return [e.strip() for e in data.strip().split(',')]

# Задание 2.
def task2(sets_names: list, p: int):
    df = pd.DataFrame(columns=['НМ', 'ЛИН', 'КИН', 'Энтропия', 'D', 'd'])
    sets_keys = list(SETS.keys())
    for set_name in SETS:
        set1 = get_set(set_name)
        if set_name in sets_names:
            df.loc[len(df.index)] = [set_name, liner_index(set1), rect_index(set1), entropy(set1),
                                     clarity_yager(set1, 1 - set1, p), fuzzy_yager(set1, 1 - set1, p)]
        else:
            df.loc[len(df.index)] = [set_name, -1, -1, -1, -1, -1]
    df = df.set_index('НМ')
    min_df = df.copy()
    del min_df['D']
    for col in min_df.columns:
        min_df[col] = np.where(min_df[col] == -1, np.inf, min_df[col])
        min_df[col] = np.where(min_df[col] == min_df[col].min(), 1, 0)
    max_df = df.copy()
    del max_df['D']
    for col in max_df.columns:
        max_df[col] = np.where(max_df[col] == max_df[col].max(), 1, 0)
    return df, min_df, max_df

#--------------------------------------------------------------------------------

def vector_indicator(set1):
    return 2 * np.abs(set1 - common_set(set1))

# Задание 3.
def task3(sets_names: list):
    cols = [ f'x{i + 1}' for i in range(get_set(sets_names[0]).shape[0]) ]
    cols.append('Вект.инд.')
    df = pd.DataFrame(columns=cols)
    df = df.set_index('Вект.инд.')
    for s in SETS:
        if s in sets_names:
            df.loc[s] = vector_indicator(get_set(s))
        else:
            df.loc[s] = -1
    return df

#--------------------------------------------------------------------------------

SETS = {}

# Сюда вставляем нечеткие множества

In [2]:
# ------------------------------------------------------------------------------
# Нужно скопировать диапозон (с названиями нечетких множеств) из Excel и вставить в блокнот.
# Потом скопировать всё из блокнота в переменную data.
# ------------------------------------------------------------------------------
data = """
A	0,52	0,85	0,5	0,04	0,62	0,8	0,76	0,66
B	0,54	0,82	0,63	0,2	0,25	0,19	0,41	0,14
C	0,07	0,89	0,4	0,23	0,05	0,05	0,09	0,12
D	0,35	0,27	0,29	0,91	0,42	0,73	0,59	0,98
E	0,95	0,73	0,1	0,29	0,33	0,11	0,95	0,29

"""
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

parse_sets(data)

# Задание 1

In [3]:
# ------------------------------------------------------------------------------
# Пары нечетких множеств задания в виде '<НМ1> и <НМ2>', разделённые запятой.
# ------------------------------------------------------------------------------
set_pairs = 'A и D, B и C, A и E'
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

df, qst = task1(parse_sets_pairs(set_pairs))
print('\n##### ОТВЕТЫ ЗАДАНИЯ 1 #####\n')
ICD.display(df)
print('\n##### ВОПРОС ЗАДАНИЯ 1 #####\n')
ICD.display(qst)


##### ОТВЕТЫ ЗАДАНИЯ 1 #####



,АЛР,ОЛР,АКР,ОКР,АЕН,ОЕН
НМ,,,,,,
A и B,-1,-1,-1,-1,-1,-1
A и C,-1,-1,-1,-1,-1,-1
A и D,2.59,0.32375,1.158663,0.409649,1.3425,0.167812
A и E,2.74,0.3425,1.074709,0.379967,1.155,0.144375
B и C,1.48,0.185,0.664831,0.235053,0.442,0.05525
B и D,-1,-1,-1,-1,-1,-1
B и E,-1,-1,-1,-1,-1,-1
C и D,-1,-1,-1,-1,-1,-1
C и E,-1,-1,-1,-1,-1,-1



##### ВОПРОС ЗАДАНИЯ 1 #####



,АЛР,АКР
НМ,,
A и B,0,0
A и C,0,0
A и D,0,1
A и E,1,0
B и C,0,0
B и D,0,0
B и E,0,0
C и D,0,0
C и E,0,0


# Задание 2

In [4]:
# ------------------------------------------------------------------------------
# Имена НМ, разделенные запятой.
sets_names = 'D, B, C'
# Степень нечеткости.
p = 4
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

values, min, max = task2(parse_sets_names(sets_names), p)
print('\n##### ОТВЕТЫ ЗАДАНИЯ 2 #####\n')
ICD.display(values)
print('\n##### ВОПРОС 2.1 ЗАДАНИЯ 2 #####\n')
ICD.display(min)
print('\n##### ВОПРОС 2.2 ЗАДАНИЯ 2 #####\n')
ICD.display(max)


##### ОТВЕТЫ ЗАДАНИЯ 2 #####



,ЛИН,КИН,Энтропия,D,d
НМ,,,,,
A,-1,-1,-1,-1,-1
B,0.55,0.594643,0.921003,0.117548,0.930105
C,0.28,0.358748,0.755427,0.165887,0.901363
D,0.53,0.593886,0.949048,0.136739,0.918695
E,-1,-1,-1,-1,-1



##### ВОПРОС 2.1 ЗАДАНИЯ 2 #####



,ЛИН,КИН,Энтропия,d
НМ,,,,
A,0,0,0,0
B,0,0,0,0
C,1,1,1,1
D,0,0,0,0
E,0,0,0,0



##### ВОПРОС 2.2 ЗАДАНИЯ 2 #####



,ЛИН,КИН,Энтропия,d
НМ,,,,
A,0,0,0,0
B,1,1,0,1
C,0,0,0,0
D,0,0,1,0
E,0,0,0,0


# Задание 3

In [5]:
# ------------------------------------------------------------------------------
# Тут ничего менять не надо. Имена НМ задаются в Задании 2.
# ------------------------------------------------------------------------------

df = task3(sets_names)
print('\n##### ОТВЕТЫ ЗАДАНИЯ 3 #####\n')
ICD.display(df)


##### ОТВЕТЫ ЗАДАНИЯ 3 #####



,x1,x2,x3,x4,x5,x6,x7,x8
Вект.инд.,,,,,,,,
A,-1,-1,-1,-1,-1,-1,-1,-1
B,0.92,0.36,0.74,0.4,0.5,0.38,0.82,0.28
C,0.14,0.22,0.8,0.46,0.1,0.1,0.18,0.24
D,0.7,0.54,0.58,0.18,0.84,0.54,0.82,0.04
E,-1,-1,-1,-1,-1,-1,-1,-1
